In [417]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import seaborn as sns
from sklearn import datasets, linear_model
import scipy.linalg as sla
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from collections import Counter
from IPython.core.display import display, HTML
sns.set_style('darkgrid')
import scipy.stats as sps

Matrix $A \in \mathbb{R}^{n\times m}$, n -- number of samples, m -- number of features\\

Vector $y\in \mathbb{R}^n$

We suppose thar $A$ and $y$ are normalized to 1

Covariance matrix $C \in \mathbb{R}^{d\times d}$

In [406]:
n = 100
m = 20
A = np.random.rand(m, n)
y = np.random.rand(n)

In [309]:
A.shape

(20, 100)

In [346]:
print(y.shape)
# print(y)

(100,)


In [338]:
# the function below returns a covariance matrix C (m x m) for columns x_1, x_2,... of matrix X (m x n)
# normalized to have expectation 0 and variation 1
def get_covariance_matrix(X):
    cov_matrix = np.cov(X, bias=True)
    return cov_matrix

In [340]:
# returns normalized vector of covariances between y and columns x_1, x_2,...of matrix X 
# 
def get_covariance_vector(X, y):
    return np.cov(np.vstack((X_T, y)))[-1,:-1]
    

In [90]:
def r2_score(y_true, y_pred, *, sample_weight=None,
             multioutput="uniform_average"):
    """R^2 (coefficient of determination) regression score function.
    Best possible score is 1.0 and it can be negative (because the
    model can be arbitrarily worse). A constant model that always
    predicts the expected value of y, disregarding the input features,
    would get a R^2 score of 0.0.
    Read more in the :ref:`User Guide <r2_score>`.
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    sample_weight : array-like of shape (n_samples,), optional
        Sample weights.
    multioutput : string in ['raw_values', 'uniform_average', \
'variance_weighted'] or None or array-like of shape (n_outputs)
        Defines aggregating of multiple output scores.
        Array-like value defines weights used to average scores.
        Default is "uniform_average".
        'raw_values' :
            Returns a full set of scores in case of multioutput input.
        'uniform_average' :
            Scores of all outputs are averaged with uniform weight.
        'variance_weighted' :
            Scores of all outputs are averaged, weighted by the variances
            of each individual output.
        .. versionchanged:: 0.19
            Default value of multioutput is 'uniform_average'.
    Returns
    -------
    z : float or ndarray of floats
        The R^2 score or ndarray of scores if 'multioutput' is
        'raw_values'.
    Notes
    -----
    This is not a symmetric function.
    Unlike most other scores, R^2 score may be negative (it need not actually
    be the square of a quantity R).
    This metric is not well-defined for single samples and will return a NaN
    value if n_samples is less than two.
    References
    ----------
    .. [1] `Wikipedia entry on the Coefficient of determination
            <https://en.wikipedia.org/wiki/Coefficient_of_determination>`_
    Examples
    --------
    >>> from sklearn.metrics import r2_score
    >>> y_true = [3, -0.5, 2, 7]
    >>> y_pred = [2.5, 0.0, 2, 8]
    >>> r2_score(y_true, y_pred)
    0.948...
    >>> y_true = [[0.5, 1], [-1, 1], [7, -6]]
    >>> y_pred = [[0, 2], [-1, 2], [8, -5]]
    >>> r2_score(y_true, y_pred,
    ...          multioutput='variance_weighted')
    0.938...
    >>> y_true = [1, 2, 3]
    >>> y_pred = [1, 2, 3]
    >>> r2_score(y_true, y_pred)
    1.0
    >>> y_true = [1, 2, 3]
    >>> y_pred = [2, 2, 2]
    >>> r2_score(y_true, y_pred)
    0.0
    >>> y_true = [1, 2, 3]
    >>> y_pred = [3, 2, 1]
    >>> r2_score(y_true, y_pred)
    -3.0
    """
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)

    if _num_samples(y_pred) < 2:
        msg = "R^2 score is not well-defined with less than two samples."
        warnings.warn(msg, UndefinedMetricWarning)
        return float('nan')

    if sample_weight is not None:
        sample_weight = column_or_1d(sample_weight)
        weight = sample_weight[:, np.newaxis]
    else:
        weight = 1.

    numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
                                                      dtype=np.float64)
    denominator = (weight * (y_true - np.average(
        y_true, axis=0, weights=sample_weight)) ** 2).sum(axis=0,
                                                          dtype=np.float64)
    nonzero_denominator = denominator != 0
    nonzero_numerator = numerator != 0
    valid_score = nonzero_denominator & nonzero_numerator
    output_scores = np.ones([y_true.shape[1]])
    output_scores[valid_score] = 1 - (numerator[valid_score] /
                                      denominator[valid_score])
    # arbitrary set to zero to avoid -inf scores, having a constant
    # y_true is not interesting for scoring a regression anyway
    output_scores[nonzero_numerator & ~nonzero_denominator] = 0.
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            # return scores individually
            return output_scores
        elif multioutput == 'uniform_average':
            # passing None as weights results is uniform mean
            avg_weights = None
        elif multioutput == 'variance_weighted':
            avg_weights = denominator
            # avoid fail on constant y or one-element arrays
            if not np.any(nonzero_denominator):
                if not np.any(nonzero_numerator):
                    return 1.0
                else:
                    return 0.0
    else:
        avg_weights = multioutput

    return np.average(output_scores, weights=avg_weights)


In [347]:
C = get_covariance_matrix(A)
print(C.shape)

(20, 20)


In [350]:
b = get_covariance_vector(A, y)
print(b.shape)

(20,)


In [355]:
# R-squared coefficient
# 
R_2 = np.dot(b.T, np.dot(LA.inv(C), b))
print(R_2)

0.019258634477752175


In [353]:
E = np.mean(y)

In [398]:
from sklearn.metrics import r2_score
n_test = 5
m_test = 3
A_test = np.random.rand(m_test, n_test)
print(A_test)
print(A_test.shape)
y_test = np.random.rand(n_test)
y_true = y_test
print(y_true, '\ny_true shape', y_true.shape)
alpha = np.random.rand(m_test)
print(alpha.shape)
y_pred = np.dot(alpha, A_test)
print(y_pred)
print(get_covariance_matrix(A_test))

[[0.19737756 0.75686821 0.87705782 0.07998631 0.81287681]
 [0.57081788 0.13171926 0.40505844 0.21287215 0.35543807]
 [0.25083931 0.68852299 0.33749733 0.15015977 0.10220112]]
(3, 5)
[0.84156902 0.08420507 0.77366252 0.87797207 0.7480465 ] 
y_true shape (5,)
(3,)
[0.69445541 1.19696823 1.22152442 0.30391545 0.97247455]
[[ 0.1127975  -0.00790294  0.02571064]
 [-0.00790294  0.02343482 -0.01473869]
 [ 0.02571064 -0.01473869  0.04323573]]


In [378]:
# r_2 = (np.mean())/()

print('sklearn r2 score is: ', r2_score(y_true, y_pred))

sklearn r2 score is:  -1.613043819880836


In [ ]:
X = np.random.rand

In [418]:
# количество признаков
n = 30
# размер выборки
m = 100
def data_generator(n, m, cov=0.2):
  # cov - коэффициент, задающий ковариацию между значениями
  # исходная мартица ковариации (может не быть положительно определенной) 
  pred_C = (1 - cov) * np.eye(n) + cov * np.ones((n, n))
  # вектор средних значений
  mean = np.zeros((n, ))
  # новая матрица ковариации, близкая к исходной, но положительно определенная
  # и с более разнообразными собственными значениями
  C = np.cov(sps.multivariate_normal(mean, pred_C).rvs(size=m).T)
  # поделим строки и стобцы на корень из диагонали чтобы генерируемые Х имели
  # единичную дисперсию
  diag_sqrt = np.sqrt(np.diag(C))
  C = C / diag_sqrt
  C = C.T / diag_sqrt
  # генерируем выборку - она имеет нулевое матожидание и (почти) единичную 
  # дисперсию в каждом столбце
  X = sps.multivariate_normal(mean, C).rvs(size=m)
  # генерируем коэффициенты и делим их на соответствующее число, чтобы в итоге
  # дисперсия Z была равна 1
  a = sps.uniform(loc=0, scale=1).rvs(n)
  a /= (a.dot(a) * (1 - cov) + cov * (a.sum())**2)
  # генерируем предсказываемую переменную так, чтобы ее дисперсия была равна 1
  # для этого делим на соответствующий коэффициент 
  Z = X.dot(a) + sps.norm(loc=0, scale=0.1).rvs(size=m)
  
  return X, Z

In [427]:
m=20
n=100
X, Z=data_generator(m, n, 0.2)
print(X.shape, y.shape)

(100, 20) (100,)


In [ ]:
# input:  X (m x n) - normalized feature matrix 
#         k - selection parameter 
#         y_true - predictor variable 

def forward_regression(X, k, y_true):
    (m,n)=X.shape
#     list of values x1,x2... indexes 
    V=range(m)
    S=[]
    used=[]
    scores=[]
    r2 = -2
    alpha = np.zeros(m)
    alpha_new = np.zeros(m)
    j_max = 0
    res_sum = 0
    
    for i in range(k):
        
        for j in V:
            if j not in S:
                
                alpha_tmp=alpha
                alpha_tmp[j]=1
                
                y_pred = np.dot(alpha_tmp, X)
            
                if r2_score(y_true, y_pred) > r2:
                    r2 = r2_score(y_true, y_pred)
                    alpha_new = alpha_tmp
                    j_max=j
                 
                
                
                
        alpha = alpha_new    
        S.append(j_max)
#         V.remove(j_max)
        
    return S, alpha

In [434]:
def forward_regression(X, k, y_true):
    (m,n)=X.shape
#     list of values x1,x2... indexes 
    V=range(m)
    S=[]
    used=[]
    scores=[]
    r2 = -2
    alpha = np.zeros(m)
    alpha_new = np.zeros(m)
    j_max = 0
    res_sum = np.zeros(n)
    
    for i in range(k):
        
        for j in V:
            if j not in S:
                y_pred = res_sum + X[j]
            
                if r2_score(y_true, y_pred) > r2:
                    r2 = r2_score(y_true, y_pred)
                    j_max=j
                 
        res_sum = res_sum + X[j_max]        
                
                
#         alpha = alpha_new    
        S.append(j_max)
#         V.remove(j_max)
        
    return S, alpha

In [435]:
A.shape
# y_true.shape
forward_regression(A, 5, Z)

([0, 0, 0, 0, 0],
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

In [387]:
alpha = np.zeros(5)
alpha[2] = 1
alpha

array([0., 0., 1., 0., 0.])